<a href="https://colab.research.google.com/github/kailliang/LLAMA-3-Local-Deploying/blob/main/LLAMA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate bitsandbytes sentencepiece
!pip install modelscope

# 💡 Core Steps:
##1️⃣ Imports

In [ ]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers.generation import GenerationConfig

##2️⃣ Deploy LLAMA-3

In [ ]:
model_dir = snapshot_download("LLM-Research/Meta-Llama-3-8B-Instruct", cache_dir='./')
model_path = "/content/LLM-Research/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
llm_chain = AutoModelForCausalLM.from_pretrained(model_path,device_map ="auto", trust_remote_code=True).eval()
llm_chain.generation_config = GenerationConfig.from_pretrained(model_path, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

##3️⃣ Build the Inference Function

In [ ]:
text = "Hello, please tell me a joke"
messages = [{"role": "user", "content": text}]
input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True,
                                          add_generation_prompt=True, return_tensors='pt')
output_ids = llm_chain.generate(input_ids.to('cuda'), max_new_tokens=4098)
response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

In [ ]:
print(response)

Here's one:

Why couldn't the bicycle stand up by itself?

(wait for it...)

Because it was two-tired!

Hope that made you smile! Do you want to hear another one?


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from huggingface_hub import snapshot_download

# Download model
# model_dir = snapshot_download("LLM-Research/Meta-Llama-3-8B-Instruct", cache_dir='./')
# model_path = "/content/LLM-Research/Meta-Llama-3-8B-Instruct"

# Load tokenizer and set pad_token_id if not set
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
# llm_chain = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     device_map="auto",
#     trust_remote_code=True).eval()

# Set generation configuration
llm_chain.generation_config = GenerationConfig.from_pretrained(model_path, trust_remote_code=True)

# Input text and conversation format
text = "Hello"
messages = [{"role": "user", "content": text}]

# Tokenize input with padding and attention mask
input_ids = tokenizer.apply_chat_template(
    conversation=messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors='pt',
    padding=True)

# Move input tensors to GPU and pass attention_mask
input_ids = input_ids.to('cuda')
attention_mask = (input_ids != tokenizer.pad_token_id).to('cuda')  # Create attention mask

# Generate response
output_ids = llm_chain.generate(
    input_ids,
    attention_mask=attention_mask,
    max_new_tokens=4098
)

# Decode the response
response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
print(response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?
